In [7]:
import cv2
import torch
from depth_anything_v2.dpt import DepthAnythingV2

import matplotlib.pyplot as plt

def display(img):
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

encoder = 'vitl' # or 'vits', 'vitb', 'vitg'

model = DepthAnythingV2(**model_configs[encoder])
model.load_state_dict(torch.load(f'../checkpoints/depth_anything_v2_{encoder}.pth', map_location='cpu'))
model = model.to(DEVICE).eval()

scene = 3
view = 0
image_path = f'/home/ryo/Dataset/forcemap/tabletop_airec241008/rgb{scene:05d}_{view:05d}.jpg'
raw_img = cv2.imread(image_path)
depth = model.infer_image(raw_img) # HxW raw depth map in numpy

depth
# display(raw_img)
# display(depth)

/tmp/ipykernel_2773276/2747308035.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'../checkpoints/depth_anything_v2_{encoder}.pth', ma

array([[  4.5881944,   4.3886976,   4.2572284, ...,   6.539229 ,
          6.531567 ,   6.8968353],
       [  4.9612384,   4.7239246,   4.695693 , ...,   6.9643345,
          7.0257616,   7.3885427],
       [  5.1125097,   5.1153655,   5.0791407, ...,   7.4590383,
          7.3869867,   7.5924773],
       ...,
       [290.68063  , 291.04498  , 291.2563   , ..., 297.63953  ,
        297.91287  , 298.27298  ],
       [291.42126  , 291.7223   , 292.38165  , ..., 298.53845  ,
        298.81622  , 298.81033  ],
       [291.74466  , 292.03473  , 293.48035  , ..., 299.63245  ,
        299.06766  , 300.2733   ]], dtype=float32)

v5の実験
・スケールは重要：vits -> vitbでかなり性能が上がった
　・vitlならさらに性能が上がるか（必ずしもそうでない．良いseedを引いただけか？）
・seedによる当たり外れが非常に大きいかも
　・同じ条件で2.9e-02くらいから中々下がらないケースもある

・architectureの検討項目
　・入力画像のNormalization：pre-trainingの分布に合わせる（1回のtrialでは悪くなった．seedの問題の方が大きそう）
　・outputのactivation：relu vs sigmoid（dataのrangeを考えるとsigmoidの方が良さそうだが，1回のtrialではreluが良かった）
　　・これは複数回試しても明らかにreluの方が良い
　・batch size：8が良さそう
　・学習率：5e-4が良さそう

・DPTの学習
　・学習率：1e-4 for decoder

・ResNet版
　・outputのactivationをreluにしても変化なし1.6e-02くらいが下限でoverfitting

・fmap_2025-01-21-20-36-32
　・vitl, input normalize, output relu

・pre-trained weightをloadできていないことに気づいた => ランダムなencoderを使っていれば，seedの影響は大きいだろう

仕切り直し
・Transformer_2025-01-21-21-13-05
　・悪くない結果（ResNetよりよくなった）
・更なる改良案
　・depth headとmulti-taskで学習させる（bp signalを増やす）
　・encoderをdinov2 weightで初期化する
　・encoderもfine-tuneする
　・decoder architectureの改良
　・data量を増やす
・注意点
　・過学習気味になっているところから，test lossが大きく減ることがある
　・
